# Your Turn!

Create the VM with the sources and all the compilers and interpreters that we need for today.

Find everything on the course's Github repository under the directory https://github.com/datsoftlyngby/soft2018fall-si-teaching-material/tree/master/lang_integration.

# Integration via Operating System Processes


This integration mechanism is the most language agnostic. As soon as you have some binary program that you can call from the CLI, you can integrate it with a simialar mechanism in the language of your choice.

![](images/li_os.png)

The code for this example can be found under `lang_integration/bin_python`.

### A Python Program Calling a Binary via a Separate OS Process

```python
import subprocess
import time


proc = subprocess.Popen(["./cli_avg", '{"matrix": [[1,2,3], [3,4,5,5]]}'], 
                  stdout=subp.PIPE, stdin=subp.PIPE)

data = None
for line in iter(proc.stdout.readline, b''):
    print('Got result: {}'.format(line))
    data = [float(el) for el in line.replace('[', '').replace(']', '').split(',')]
    print([2 * el for el in data])
proc.communicate()
```

### A Nim CLI Program 

```nim
import os
import json
import math
import sequtils
import strutils

type
  Data = object
    matrix: seq[seq[int]]
  
proc main(): seq[float] =
    if paramCount() != 1:
        quit("Usage: " & getAppFilename() & " {\"matrix\": [[1,2,3], [3,4,5,5]]}")
    else:
        let matrixStr = paramStr(1)
        let jsonNode = parseJson(matrixStr)
        let data = json.to(jsonNode, Data)

        var res: seq[float] = @[]
        for vector in data.matrix:
            res.add(sum(vector) / vector.len)

        return res

let valSeq = main()
let strings = valSeq.mapIt(string, $it).join(",")
echo "[" & strings & "]"
```

After compiling the program above with 

```bash
$ nim compile cli_avg.nim
```

you have a binary in the local directory, which is called `cli_avg`.

You can call that directly, e.g., via `./cli_avg '{"matrix": [[1,2,3], [3,4,5,5]]}'`.

That is, the same as the above Python program does to interact with the binary:

```bash
$ python call_bin.py
Got result: [2.0,4.25]

[4.0, 8.5]
```


# Your turn!

Take the above `cli_avg` (Linux) binary and call it from the language of your choice. Alternatively, call another CLI program, such as `hostname -I` from the language of your choice and to receive the external IP of your machine.





## Student Solutions 2017

### Java

```java
// Java
public static void main(String[] args) {
   Main main = new Main();

        System.out.println(main.executeShellCmd("ipconfig"));

    }

    private String executeShellCmd(String command) {
        StringBuffer output = new StringBuffer();
        Process p;

        try {
            p = Runtime.getRuntime().exec(command);
            p.waitFor();
            BufferedReader reader = new BufferedReader(new InputStreamReader(p.getInputStream()));
            String line = "";

            while((line = reader.readLine()) != null) {
                output.append(line + "\n");
            }
        } catch (Exception e) {
            e.printStackTrace();
        }

        return output.toString();
    }
}
```

### Javascript

```javascript
var exec = require('child_process').exec;
var child;

child = exec("hostname -I", function (error, stdout, stderr) {
    console.log("\"Hostname -I\" --- " + stdout)
    if (error !== null) {
        console.log('exec error: ' + error);
    }
});
```

### Python

```python
import subprocess

p = subprocess.Popen('hostname -i', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print line,
retval = p.wait()
```

```python
import os
cmd = 'hostname -I'
os.system(cmd) 
```


### C-Sharp

```csharp
static void Main(string[] args)
        {

            Process pro = new Process();
            ProcessStartInfo startInfo = new ProcessStartInfo();
            startInfo.UseShellExecute = false;
            startInfo.RedirectStandardOutput = true;
            startInfo.FileName = "CMD.exe";
            startInfo.Arguments = "/c hostname";
            pro.StartInfo = startInfo;
            pro.Start();
            string output = pro.StandardOutput.ReadLine();
            Console.WriteLine(output);
            Console.ReadKey();
        }
```

### Perl

```
perl -e 'exec("hostname");'
```
--------


## Student Solutions 2018

## Running binary from Nim:

```nim
import osproc

proc runCliAvg(): int =
    let (outp, errorCode) = execCmdEx("../bin_from_python/cli_avg \"{\"matrix\": [[1,2,3], [3,4,5]]}\"")
    
    if errorCode != 0:
        echo errorCode
    else:
        echo outp
    return errorCode


when isMainModule:
    discard runCliAvg()
```

### C
```c
#include <stdlib.h>

int main()
{
    system("../bin_from_python/cli_avg \"{\"matrix\": [[1,2,3], [3,4,5]]}\"");
    return 0;
}
```

--------

# Compilation to the Same Level of Abstraction

![](images/li_same_abstraction.png)

The code for this example can be found under `lang_integration/groovy_from_java`.

## JVM Bytecode for Integration
For exmple interaction in between Java and Groovy, both languages compiling to JVM bytecode.


### A Groovy Script

```Groovy
def computeAvg(matrix) {
	println matrix
	def result = []
	matrix.eachWithIndex { a, i ->
  		result.add(a.sum() / a.size())
	}
	return result	
}

def matrix = [[1, 23, 5, 4], [2, 4, 5, 6, 7, 8]]
println computeAvg(matrix)
```

Since the Groovy program is self-contained, you can run it with: `groovy Script.groovy`.


### A Java Program Calling a Groovy Script

```Java
public class Main {
    public static void main(String[] args) {

		int[][] matrix = new int[][]{
		  { 1, 2, 3, 4, 5 },
		  { 9, 8, 7, 6, 5, 4, 3, 1 },
		};
    
        Script script = new Script();
        System.out.println(script.computeAvg(matrix));
    }
}
```

### Compiling the Sources


After compiling the sources with the following commands...

```bash
$ groovyc Script.groovy
$ javac -classpath .:$GROOVY_HOME/embeddable/groovy-all-2.4.12.jar Main.java
```

you will find three `.class` files

```bash
$ ls -ltrh *.class
-rw-r--r-- 1 vagrant vagrant 2.7K Dec 11 14:33 Script$_computeAvg_closure1.class
-rw-r--r-- 1 vagrant vagrant 3.0K Dec 11 14:33 Script.class
-rw-r--r-- 1 vagrant vagrant  580 Dec 11 14:35 Main.class
```

What is in those `.class` files? Check it for example with `javap -c Main.class`, `javap -c Script.class`, etc.

### Running the Sources

Now, you can execute the compiled program on the JVM with

```bash
java -classpath .:$GROOVY_HOME/embeddable/groovy-all-2.4.12.jar Main
```

_Reference:_ This section is based on the following Stackoverflow entry https://stackoverflow.com/a/3990829

### Your Turn!


What is in those `.class` files? 

Check their contents for example with `javap -c Main.class`, `javap -c Script.class`, etc. and describe to your neighbour what you see/observe.


## Object Code as Common Level of Integration

Of course you can integrate on any level of abstraction. Above, we integrated our programs on JVM bytecode level. We could do the same on object code level. 

![](images/li_same_abstraction.png)

The code for this example can be found under `lang_integration/ada_from_c`.

Consider the two following Ada packages, whose functions get called in a C `main` program.

```Ada
package Unit1 is
   function Add (A, B : Integer) return Integer;
   pragma Export (C, Add, "add");
end Unit1;
```

```Ada
package body Unit1 is
   function Add (A, B : Integer) return Integer is
   begin
      return A + B;
   end Add;
end Unit1;
```


```Ada
package Unit2 is
   function Sub (A, B : Integer) return Integer;
   pragma Export (C, Sub, "sub");
end Unit2;
```

```Ada
package body Unit2 is
   function Sub (A, B : Integer) return Integer is
   begin
      return A - B;
   end Sub;
end Unit2;
```

Now, we want to call the two functions from above from C main program.

```C
#include <stdio.h>

extern void adainit (void);
extern void adafinal (void);
extern int add (int, int);
extern int sub (int, int);

int main (int argc, char *argv[])
{
   int a = 21, b = 7;

   adainit();

   /* Should print "21 + 7 = 28" */
   printf("%d + %d = %d\\n", a, b, add(a, b));

   /* Should print "21 - 7 = 14" */
   printf("%d - %d = %d\\n", a, b, sub(a, b));

   adafinal();
}
```

Now, you can compile the program by linking the object files together.

```bash
$ gcc -c main.c
$ gnatmake -c unit1.adb
gcc-4.9 -c unit1.adb
$ gnatmake -c unit2.adb
gcc-4.9 -c unit2.adb
$ gnatbind -n unit1.ali unit2.ali
$ gnatlink unit2.ali main.o -o main
$ ls -ltrh
total 64K
-rw-r--r-- 1 vagrant vagrant  111 Nov 22  2017 unit1.ads
-rw-r--r-- 1 vagrant vagrant  125 Nov 22  2017 unit1.adb
-rw-r--r-- 1 vagrant vagrant  111 Nov 22  2017 unit2.ads
-rw-r--r-- 1 vagrant vagrant  125 Nov 22  2017 unit2.adb
-rw-r--r-- 1 vagrant vagrant  395 Nov 22  2017 main.c
-rw-r--r-- 1 vagrant vagrant  233 Jan 19  2018 Makefile
-rw-r--r-- 1 vagrant vagrant 1.9K Dec 11 15:08 main.o
-rw-r--r-- 1 vagrant vagrant 1.3K Dec 11 15:08 unit1.o
-rw-r--r-- 1 vagrant vagrant  387 Dec 11 15:08 unit1.ali
-rw-r--r-- 1 vagrant vagrant 1.3K Dec 11 15:08 unit2.o
-rw-r--r-- 1 vagrant vagrant  387 Dec 11 15:08 unit2.ali
-rwxr-xr-x 1 vagrant vagrant  17K Dec 11 15:08 main
```


_Reference:_ This section is based on the GNAT User Manual https://gcc.gnu.org/onlinedocs/gnat_ugn/Interfacing-to-C.html

### How do I see the contents of a binary?

```bash
$ xxd <file>
```

```bash
$ objdump -D -b binary -m i8086 <file>
```

# Your turn!


Try the above type of integration in the language/technology/framework of choice. Protocoll your steps in a script or a `Makefile`.

  * **JVM** Call for example a bit of Kotlin code from Java
  * **.Net** Try for example to call IronPython from C#, F# from C#


---------------

## Student Solutions

### Java & Kotlin
```java
// JAVA FILE
package test;
public class Hello {

    public static void main(String[] args) {
        System.out.println("Hi from Java");
    }
}
```

```bash
// EXECUTE IN TERMINAL
javac Hello.java
```

```kotlin
// KOTLIN FILE
import test.Hello

fun main(args: Array<String>) {
    println(Hello.main(null))
}
```

----------------

# Integration via Shared Libraries

![](images/li_so.png)

The code for this example can be found under `lang_integration/cpp_so_from_ruby`.

Let's say we have a bit of C++ code, which we want to make accesible from another language, which knows how to call shared libraries (`.so`/`.dll`) files.

Below, we have a C++ function which can compute the line-wise average of a 2D-numerical array. First the header file `code.h`:

```cpp
#include <vector>

std::vector<double> average (std::vector< std::vector<double> > i_matrix);
```


and the actual implementation in a file called `code.cpp`
```cpp
#include <vector>
#include "code.h"

using namespace std;

vector<double> average (vector< vector<double> > i_matrix) {

  // Compute average of each row..
  vector <double> averages;
  for (int r = 0; r < i_matrix.size(); r++){
    double rsum = 0.0;
    double ncols= i_matrix[r].size();
    for (int c = 0; c < i_matrix[r].size(); c++){
      rsum += i_matrix[r][c];
    }
    averages.push_back(rsum/ncols);
  }
  return averages;
}
```

### A SWIG Interface

With a tool called SWIG (http://www.swig.org), we can declare which parts of that code should be accesible, e.g., to Ruby and build a shared library out of it.

#### SWIG?

  > SWIG is a software development tool that connects programs written in C and C++ with a variety of high-level programming languages. SWIG is used with different types of target languages including common scripting languages such as Javascript, Perl, PHP, Python, Tcl and Ruby. The list of supported languages also includes non-scripting languages such as C#, ..., D, Go language, Java including Android, Lua, Modula-3, OCAML, Octave, Scilab and R.
  > http://www.swig.org

SWIG requires an interface file such as the following `code.i`:

```swig
%module code
%{
#include "code.h"
%}
%include "std_vector.i"
namespace std {

  /* On a side note, the names VecDouble and VecVecdouble can be changed, but the order of first the inner vector matters! */
  %template(VecDouble) vector<double>;
  %template(VecVecdouble) vector< vector<double> >;
}

%include "code.h"
```


### Building the Code

#### First Using A Ruby Helper for Building a `Makefile`

The following Ruby code in a file called `extconf.rb` will create a `Makefile` which we can use for building the C++ code into a shared library:

```ruby
require 'mkmf'
system('swig -c++ -ruby code.i') or abort
create_makefile('code')
```

### Create a Wrapper and a `Makefile`

```bash
$ ruby extconf.rb  # Create wrapper and Makefile
$ make             # Compile to code.so
```


### Running the Code


In case you have the file `cli_avg.rb` with the following code

```ruby
p Code::average([[1,2,3],[4,5,6,7]])
```

you can run it by adding the library directly from the CLI:

```bash
$ ruby -r./code cli_avg.rb
```

Similarly, for running the interactive shell you have to add the link to the shared librery too:

```bash
$ irb -r./code     # try it
```


_References:_

This section is based on the following two sources:
  * https://stackoverflow.com/a/23865947
  * http://www.davidegrayson.com/presentations/20140910-swig-ruby/swig-ruby.pdf
  * https://stackoverflow.com/a/9688536


## Using SWIG Manually

The code for this example can be found under `lang_integration/cpp_so_from_python`.

With an interface file, you have to tell SWIG, which parts of your C/C++ code you want to make accessible to another language.

```swig
%module code
%{
#include "code.h"
%}
%include "std_vector.i"
namespace std {

  /* On a side note, the names VecDouble and VecVecdouble can be changed, but the order of first the inner vector matters! */
  %template(VecDouble) vector<double>;
  %template(VecVecdouble) vector< vector<double> >;
}

%include "code.h"
```

Below is the C++ interface and implementation of the code, which we want to call from Python as a normal module.

```cpp
#ifndef _code
#define _code

#include <vector>

std::vector<double> average (std::vector< std::vector<double> > i_matrix);

#endif
```

```cpp
#include <vector>
#include "code.h"

using namespace std;

vector<double> average (vector< vector<double> > i_matrix) {

  // Compute average of each row..
  vector <double> averages;
  for (int r = 0; r < i_matrix.size(); r++){
    double rsum = 0.0;
    double ncols= i_matrix[r].size();
    for (int c = 0; c< i_matrix[r].size(); c++){
      rsum += i_matrix[r][c];
    }
    averages.push_back(rsum/ncols);
  }
  return averages;
}
```


### Building the Code into a Shared Library, which can be imported in Python


```bash
$ ls -ltrh
total 20K
-rw-r--r-- 1 vagrant vagrant 315 Nov 21  2017 code.i
-rw-r--r-- 1 vagrant vagrant 436 Nov 21  2017 code.cpp
-rw-r--r-- 1 vagrant vagrant 131 Nov 21  2017 code.h
-rw-r--r-- 1 vagrant vagrant 100 Dec 12  2017 cli_avg.py
-rw-r--r-- 1 vagrant vagrant 356 Dec 12  2017 Makefile
$ g++ -c -fPIC code.cpp
$ ls -ltrh
total 56K
-rw-r--r-- 1 vagrant vagrant 315 Nov 21  2017 code.i
-rw-r--r-- 1 vagrant vagrant 436 Nov 21  2017 code.cpp
-rw-r--r-- 1 vagrant vagrant 131 Nov 21  2017 code.h
-rw-r--r-- 1 vagrant vagrant 100 Dec 12  2017 cli_avg.py
-rw-r--r-- 1 vagrant vagrant 356 Dec 12  2017 Makefile
-rw-r--r-- 1 vagrant vagrant 34K Dec 11  2018 code.o
$ swig -c++ -python code.i
$ ls -ltrh
total 436K
-rw-r--r-- 1 vagrant vagrant  315 Nov 21  2017 code.i
-rw-r--r-- 1 vagrant vagrant  436 Nov 21  2017 code.cpp
-rw-r--r-- 1 vagrant vagrant  131 Nov 21  2017 code.h
-rw-r--r-- 1 vagrant vagrant  100 Dec 12  2017 cli_avg.py
-rw-r--r-- 1 vagrant vagrant  356 Dec 12  2017 Makefile
-rw-r--r-- 1 vagrant vagrant  34K Dec 11  2018 code.o
-rw-r--r-- 1 vagrant vagrant 365K Dec 11  2018 code_wrap.cxx
-rw-r--r-- 1 vagrant vagrant  11K Dec 11  2018 code.py
$ g++ -c -fPIC code_wrap.cxx  -I/usr/include/python2.7 -I/usr/lib/python2.7
$ ls -ltrh
total 1.1M
-rw-r--r-- 1 vagrant vagrant  315 Nov 21  2017 code.i
-rw-r--r-- 1 vagrant vagrant  436 Nov 21  2017 code.cpp
-rw-r--r-- 1 vagrant vagrant  131 Nov 21  2017 code.h
-rw-r--r-- 1 vagrant vagrant  100 Dec 12  2017 cli_avg.py
-rw-r--r-- 1 vagrant vagrant  356 Dec 12  2017 Makefile
-rw-r--r-- 1 vagrant vagrant  34K Dec 11  2018 code.o
-rw-r--r-- 1 vagrant vagrant 365K Dec 11  2018 code_wrap.cxx
-rw-r--r-- 1 vagrant vagrant  11K Dec 11  2018 code.py
-rw-r--r-- 1 vagrant vagrant 665K Dec 11  2018 code_wrap.o
$ g++ -shared -Wl,-soname,_code.so -o _code.so code.o code_wrap.o
$ ls -ltrh
total 1.6M
-rw-r--r-- 1 vagrant vagrant  315 Nov 21  2017 code.i
-rw-r--r-- 1 vagrant vagrant  436 Nov 21  2017 code.cpp
-rw-r--r-- 1 vagrant vagrant  131 Nov 21  2017 code.h
-rw-r--r-- 1 vagrant vagrant  100 Dec 12  2017 cli_avg.py
-rw-r--r-- 1 vagrant vagrant  356 Dec 12  2017 Makefile
-rw-r--r-- 1 vagrant vagrant  34K Dec 11  2018 code.o
-rw-r--r-- 1 vagrant vagrant 365K Dec 11  2018 code_wrap.cxx
-rw-r--r-- 1 vagrant vagrant  11K Dec 11  2018 code.py
-rw-r--r-- 1 vagrant vagrant 665K Dec 11  2018 code_wrap.o
-rwxr-xr-x 1 vagrant vagrant 464K Dec 11  2018 _code.so
```

If you now had a Python program, called `cli_avg.py`, with the following code:

```python
import code


a = [[3, 5, 7], [8, 10, 12, 13]]
b = code.average(a)
print(a, b)
```

You could run it via `python cli_avg.py`. Here you have to use Python 2.7 as we were building the shared library for that version (`g++ -c -fPIC code_wrap.cxx  -I/usr/include/python2.7 -I/usr/lib/python2.7`) 

_Reference:_ This section is based on https://stackoverflow.com/a/23865947

# Your turn!


Use the aboce C++ code together with SWIG to create a language binding for the language of your choice.

Use the official SWIG documentation at http://www.swig.org/Doc3.0/SWIGDocumentation.html.

# Integration by Transpilation

![](images/li_trans.png)

The code for this example can be found under `lang_integration/fortran77_to_c`.

### Modernising an old Fortran Program...


The following Fortran 77 program is taken from here: https://en.wikibooks.org/wiki/Fortran/Fortran_examples#FORTRAN_77

It finds the greatest common divisor read from user input.

```Fortran
*     euclid.f (FORTRAN 77)
*     Find greatest common divisor using the Euclidean algorithm

      PROGRAM EUCLID
        PRINT *, 'A?'
        READ *, NA
        IF (NA.LE.0) THEN
          PRINT *, 'A must be a positive integer.'
          STOP
        END IF
        PRINT *, 'B?'
        READ *, NB
        IF (NB.LE.0) THEN
          PRINT *, 'B must be a positive integer.'
          STOP
        END IF
        PRINT *, 'The GCD of', NA, ' and', NB, ' is', NGCD(NA, NB), '.'
        STOP
      END

      FUNCTION NGCD(NA, NB)
        IA = NA
        IB = NB
    1   IF (IB.NE.0) THEN
          ITEMP = IA
          IA = IB
          IB = MOD(ITEMP, IB)
          GOTO 1
        END IF
        NGCD = IA
        RETURN
      END
```

As you can see in the list of generated files, `fort77` -which relies on `f2c` for the actual conversion into C code-, creates a C program (`GCD.c`). This is in turn compiled and linked into the `gcd` binary.

```bash

$ ls -ltrh
total 16K
-rwxr-xr-x 1 vagrant vagrant  737 Nov 22  2017 GCD.f
-rw-r--r-- 1 vagrant vagrant  166 Nov 22  2017 main.c
-rw-r--r-- 1 vagrant vagrant 1.6K Nov 22  2017 main.o
-rw-r--r-- 1 vagrant vagrant   73 Dec 12  2017 Makefile
$ fort77 GCD.f -o gcd
   MAIN euclid:
   ngcd:
$ ls -ltrh
total 28K
-rwxr-xr-x 1 vagrant vagrant  737 Nov 22  2017 GCD.f
-rw-r--r-- 1 vagrant vagrant  166 Nov 22  2017 main.c
-rw-r--r-- 1 vagrant vagrant 1.6K Nov 22  2017 main.o
-rw-r--r-- 1 vagrant vagrant   73 Dec 12  2017 Makefile
-rwxr-xr-x 1 vagrant vagrant 9.5K Dec 11  2018 gcd
$ ./gcd
 A?
12
 B?
36
 The GCD of 12 and 36 is 12.
```

# Integrating with other Technological Domains

![](images/li_trans.png)

The code for this example can be found under `lang_integration/nim_to_js`.

```html
<html>
  <body>
    <script type="text/javascript">
    function addTwoIntegers(a, b)
    {
      return a + b;
    }
    </script>
    <script type="text/javascript" src="calculator.js"></script>
  </body>
</html>
```

### Nim (C) Code calling a JavaScript Function

```nim
proc addTwoIntegers(a, b: int): int {.importc.}

when isMainModule:
  echo addTwoIntegers(3, 7)
```

### Building the Code

```bash
$ ls -ltrh
total 12K
-rw-r--r-- 1 vagrant vagrant  96 Nov 22  2017 calculator.nim
-rw-r--r-- 1 vagrant vagrant 215 Nov 22  2017 host.html
-rw-r--r-- 1 vagrant vagrant  73 Dec 11  2018 Makefile
$ nim js -o:calculator.js calculator.nim
Hint: used config file '/home/vagrant/nim-0.17.2/config/nim.cfg' [Conf]
Hint: system [Processing]
Hint: calculator [Processing]
Hint: operation successful (5475 lines compiled; 0.030 sec total; 3.758MiB peakmem; Debug Build) [SuccessX]
$ ls -ltrh
total 16K
-rw-r--r-- 1 vagrant vagrant   96 Nov 22  2017 calculator.nim
-rw-r--r-- 1 vagrant vagrant  215 Nov 22  2017 host.html
-rw-r--r-- 1 vagrant vagrant   73 Dec 11  2018 Makefile
-rw-r--r-- 1 vagrant vagrant 2.7K Dec 11  2018 calculator.js
```

![](images/nim_js.png)

### Integration via AST-rewriting

![](images/li_ast_rewrite.png)

The code for this example can be found under `lang_integration/hy_to_python`


#### Hylang http://hylang.org

```lisp
(defn simple-conversation []
   (print "Hello!  I'd like to get to know you.  Tell me about yourself!")
   (setv name (raw-input "What is your name? "))
   (setv age (raw-input "What is your age? "))
   (print (+ "Hello " name "!  I see you are "
              age " years old.")))

(simple-conversation)
```

#### To Python
```bash
$ hy example.hy
Hello!  I'd like to get to know you.  Tell me about yourself!
What is your name? Bumm
What is your age? 4
Hello Bumm!  I see you are 4 years old.
$ hy2py example.hy
from hy.core.language import name


def simple_conversation():
    print(u"Hello!  I'd like to get to know you.  Tell me about yourself!")
    name = raw_input(u'What is your name? ')
    age = raw_input(u'What is your age? ')
    return print(u'Hello ' + name + u'!  I see you are ' + age + u' years old.'
        )


simple_conversation()
```


# Embedding an other Language as String


![](images/li_embed.png)

### SQL Embedded in Java

```java
package dk.cphbusiness.db.neo4j.intro;

import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.SQLException;
import java.sql.Statement;


public class ConnectionTest {

    public static void main(String[] args) {
        Connection con = null;
        Statement st = null;
        ResultSet rs = null;

        String url = "jdbc:mysql://localhost:3306/your_db";
        String user = "root";
        String password = "pwd";

        String query = "SELECT * FROM your_db.t_user";

        try {

            con = DriverManager.getConnection(url, user, password);

            st = con.createStatement();
            rs = st.executeQuery(query);

            if (rs.next()) {                
                System.out.println(rs.getString(1));
            }

        } catch (SQLException ex) {
            System.err.println(ex.getMessage());
            System.err.println(ex);
        } finally { 
            try {
                if (rs != null) {
                    rs.close();
                }
                if (st != null) {
                    st.close();
                }
                if (con != null) {
                    con.close();
                }
            } catch (SQLException ex) {
                System.err.println(ex.getMessage());
                System.err.println(ex);
            }
        }
    }
}
```

### Cypher Embedded in Java


```java
package dk.cphbusiness.db.neo4j.intro;

import org.neo4j.driver.v1.*;

public class ConnectionTest {

    public static void main(String[] args) {
        Driver driver = GraphDatabase.driver( 
                "bolt://localhost:7687", 
                AuthTokens.basic( "neo4j", "class" ) );
        Session session = driver.session();

        // Run a query matching all nodes
        StatementResult result = session.run( 
                "MATCH (s)" +
                "RETURN s.name AS name, s.job AS job");

        while ( result.hasNext() ) {
            Record record = result.next();
            System.out.println( record.get("name").asString() );
        }

        session.close();
        driver.close();
    }
}
```

--------------------------

# Embedding another Language's Interpreter Directly

![](images/li_embed_inter.png)


Some languages are design as extension languages. Lua (http://www.lua.org) is one of such. Usually, you embed the entire Lua interpreter via a library into your program.

The code for this example can be found under `lang_integration/lua_from_c`.


In the following a possible Lua script, which is interpreted as a configuration file by the C program below.

```lua
-- configuration file for program `pp'
if getenv("DISPLAY") == ":0.0" then
  width = 300; height = 300
else
  width = 200; height = 200
end
```
------------
```c
#include <stdio.h>
#include "lua.h"
#include "lauxlib.h"
#include "lualib.h"

void load (char *filename, int *width, int *height) {
  lua_State *L = lua_open();
  luaopen_base(L);
  luaopen_io(L);
  luaopen_string(L);
  luaopen_math(L);

  if (luaL_loadfile(L, filename) || lua_pcall(L, 0, 0, 0))
    error(L, "cannot run configuration file: %s",
             lua_tostring(L, -1));

  lua_getglobal(L, "width");
  lua_getglobal(L, "height");
  if (!lua_isnumber(L, -2))
    error(L, "`width' should be a number\n");
  if (!lua_isnumber(L, -1))
    error(L, "`height' should be a number\n");
  *width = (int)lua_tonumber(L, -2);
  *height = (int)lua_tonumber(L, -1);

  lua_close(L);
}

int main() {
    int width = 10;
    int height = 5;
    int *width_p = &width;
    int *height_p = &height;
    load("config.lua", width_p, height_p);
    printf("%d, %d\\n", *width_p, *height_p);
    return 0;
}
```

_Reference:_ This section is based on http://www.lua.org/pil/25.html


# Your turn!

Write and **build** a C program with the code from above that consumes the Lua configuration script and prints the product of the two variables `width` and `height`. The product shall be computed in the C host program.

--------------------------

In [ ]:
gcc -I/usr/include/lua5.3 -ansi -pedantic -Wall -Wextra -O2 pp.c
